In [44]:
from brainpipe.system import study
from os.path import join, exists
from os import makedirs
import numpy as np
from itertools import product
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
from scipy.stats import sem

st = study('Olfacto')
exp, pval, met = 'R', '0.01', 'zFisher'
pathdata = join(st.path, 'figure/TPSim_LDA_'+exp+'_by_cond_6freqs_3s_dissim/')
# pathdata = join(st.path, 'figure/0_clf_pow_sklearn_time_'+exp+'_by_cond/')

leg = ['Poor', 'Rich']
freqs = ['theta']
#freqs = ['delta','theta', 'alpha', 'beta','low_gamma','high_gamma']
#freqs = ['raw']

#dfname = '2_all_subjects_info_elecs_AUC_by_roi_'+pval+'_theta_corr.csv'
dfname = '2_all_subjects_info_elecs_AUC_Signif_th_'+pval+'_mean_corr_all_11_not_corr.csv'
df = pd.read_csv(pathdata+dfname)
df_sel = df
df_ = df_sel[['subjects','s_Mai_RL','channels','x','y','z','elecs_num']]

#Select results significant for each frequency band and ROIs
for freq in freqs:
    df_freq = pd.concat([df_,df_sel.filter(like=freq[:])], axis=1)
    #print(df_freq)
    df_sig = df_freq.loc[df_freq['sig_1lev'+freq[:]]>0]
    sig_rois = np.unique(df_sig['s_Mai_RL'])
    print(freq, sig_rois)
    for roi in sig_rois:
        df_plot = None
        df_roi_f = df_sig.loc[df_sig['s_Mai_RL'] == roi]
        #print(df_roi_f)
        nb_su = len(np.unique(df_roi_f['subjects']))
        #if roi == roi_sel and freq == freq_sel:
        nb_elec_su = Counter(df_roi_f['subjects'])
        if (nb_su >= 3 and roi != 'OFC') or (nb_su >=2 and roi in ['pPirT','OFC_olf']):
            print('\n>>> for %s in %s, %s subjects have sig elecs' % (roi,freq[:],nb_su))
#             print('organized as follow', nb_elec_su)
            auc = df_roi_f[freq+'_AUC'].values
            auc_mean, std_auc = np.mean(auc), np.std(auc)
            print('AUC decoding score is %s in average +/- %s' %(round(auc_mean,2),
                                round(std_auc,2)))
            #create a value of 1 for increase, -1 for decrease and 0 for no change
            df_roi_f['sign'] = np.sign(df_freq[freq+'_Pow1'] - df_freq[freq+'_Pow0'])
            inc = (df_roi_f.loc[df_roi_f.sign == 1.0]).shape[0]
            dec = (df_roi_f.loc[df_roi_f.sign == -1.0]).shape[0]
            print('%s electrodes showed increased TPSim, while %s showed decrease out of %s elecs' 
                  % (inc,dec,inc+dec))
            df_inc = df_roi_f.loc[df_roi_f.sign == 1.0].groupby(['subjects']).count()
            df_dec = df_roi_f.loc[df_roi_f.sign == -1.0].groupby(['subjects']).count()
            print('NB of subjects with increase',df_inc.shape[0])
            print('NB of subjects with decrease',df_dec.shape[0])
            
            if (df_inc.shape[0] >= 3) or (df_inc.shape[0] >=2 and roi in ['pPirT','OFC_olf']):
                df_plot = df_roi_f.loc[df_roi_f.sign == 1.0]
                n_subj = df_inc.shape[0]
                print(df_plot)
            if (df_dec.shape[0] >= 3) or (df_dec.shape[0] >=2 and roi in ['pPirT','OFC_olf']):
                df_plot = df_roi_f.loc[df_roi_f.sign == -1.0]
                n_subj = df_dec.shape[0]
                print(df_plot)
            
            if df_plot is not None:
                if not exists(pathdata+'npy_figs/'):
                    makedirs(pathdata+'npy_figs/')
                    
                x = [np.mean(df_plot[freq+'_Pow0']),np.mean(df_plot[freq+'_Pow1'])]        
                sd = [np.std(df_plot[freq+'_Pow0']),np.std(df_plot[freq+'_Pow1'])]        
                title = '{} in {} at {} (p={}) \n {} elecs in {} patients'.format(
                                roi,freq,exp,pval,df_plot.shape[0],n_subj)
                plt.title(title)
                plt.bar(np.arange(len(leg)), x, width=0.5, yerr=sd)
                plt.ylabel('Similarity (r)')
                plt.xticks(np.arange(len(leg)), labels=leg)
                plt.savefig(pathdata+'npy_figs/{}_{}_{}_{}_tps.png'.format(freq,roi,exp,pval))
                plt.savefig(pathdata+'npy_figs/{}_{}_{}_{}_tps.pdf'.format(freq,roi,exp,pval))
                df_plot.to_csv(pathdata+'npy_figs/{}_{}_{}_{}_tps.csv'.format(freq,roi,exp,pval))
                plt.clf()
                plt.close()
                del df_plot

-> Olfacto loaded
theta ['ACC' 'IFG' 'Ins' 'MFG' 'OFC' 'OFC_olf' 'PHG' 'SFG' 'aHC' 'pHC' 'pPirT']

>>> for IFG in theta, 4 subjects have sig elecs
AUC decoding score is 0.7 in average +/- 0.03
6 electrodes showed increased TPSim, while 2 showed decrease out of 8 elecs
NB of subjects with increase 4
NB of subjects with decrease 2
    subjects s_Mai_RL   channels      x      y      z  elecs_num  theta_AUC  \
97      FERJ      IFG      o9-o8  34.30  43.00 -14.60         40   0.694048   
163     PIRJ      IFG    o11-o10  44.30  39.10 -13.05         25   0.672222   
164     PIRJ      IFG    o12-o11  48.10  39.20 -13.15         26   0.662500   
186     SEMC      IFG      j3-j2  37.90  24.90 -22.70         18   0.755076   
257     VACJ      IFG  k'12-k'11 -43.35  33.05   9.50         27   0.684864   
277     VACJ      IFG  o'12-o'11 -40.90  36.30 -10.65         47   0.678798   

     theta_Pow0  theta_Pow1  sig_1levtheta  sign  
97     0.534688    0.882475              1   1.0  
163    0.6306

In [22]:
freqs = ['0_theta', '1_alpha', '2_beta', '3_gamma']
su_odor_groups = {'CHAF' : {'low':['1','2','4','5'],
                              'high':['3','8','7','9']},
                    'VACJ' : {'low':['11','14','12','10'],
                              'high':['15','17','16','13']},
                    'SEMC' : {'low':['7','10','11','12','13'],
                              'high':['5','8','9']},
                    'PIRJ' : {'low':['1','9','5'],
                              'high':['4','6','7','18']},
                    'LEFC' : {'low':['15','2','1','16'],
                              'high':['14','3','4','17']},
                    'FERJ' : {'low':['7','2','16','17'],
                              'high':['12','1','5','13']},}
context_su = {'CHAF': {'M':[5,7,8,9],'F':[1,2,3,4]},
              'LEFC': {'M':[1,2,3,4],'F':[14,15,16,17]},
              'PIRJ': {'M':[4,9,1,18],'F':[6,5,7]}, #missing odor 15
              'VACJ': {'M':[14,15,16,17],'F':[10,11,12,13]},
              'SEMC': {'M':[10,11,12,13],'F':[5,7,8,9]},
              'FERJ': {'M':[16,17,5,7],'F':[12,13,2,1]},}

In [25]:
"""
Plot TPSim matrices (RDM) for all odors by subject 
Plot Graph 2D representations of RDM matrices
"""
from brainpipe.system import study
from os.path import join
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib import cm
import seaborn as sns 
from matplotlib.patches import Patch
from sklearn.manifold import MDS
import glob

###############################################################################
st = study('Olfacto')
pathdata = join(st.path, 'figure/TPSim_LDA_{}_by_cond_6freqs_3s_dissim/npy_figs/')
path_npz = join(st.path,'feature/TPSim_{}_By_Odor_By_Cond/')
path_pow = join(path_npz, 'similarity_matrix_btw/TPS_pearson_{}_{}_btw_odors_mean.npz')
savename = join(pathdata, 'rdm_{}/{}_{}_{}_{}_{}_p={}_rdm.png')
savename2 = join(pathdata, 'rdm_{}/{}_{}_{}_{}_{}_p={}_rdm.pdf')
###############################################################################
exp, pval = 'Ret', '0.01' #Ret, Enc
freqs = ['0_theta']#, '1_alpha', '2_beta', '3_gamma']
###############################################################################

files = glob.glob(pathdata.format(exp[0])+freqs[0][2:]+'*'+pval+'_tps.csv')

for fi in files:
    roi = fi.split('_')[13]
    df = pd.read_csv(fi)
    subjects, channels = df['subjects'], df['channels']
    
    for su, chan in zip(subjects,channels):
        mat = np.load(path_pow.format(exp,su,freqs[0]),allow_pickle=True)
        mat_chans = mat['channels']
        elec_id = [i for i,c in enumerate(mat_chans) if c == chan]
        combs, tps, chan_test = mat['comb'], mat['tps'][elec_id,:], mat_chans[elec_id]

        n_od = len(np.unique(combs))
        idx = list([combs[0,0]])+list([o for o in combs[:n_od-1,1]])
        tri = np.zeros((n_od, n_od))
        tri[np.triu_indices(n_od, 1)] = tps
        tri[np.tril_indices(n_od, -1)] = tri.T[np.tril_indices(n_od, -1)]
        model = MDS(n_components=2, dissimilarity='precomputed', random_state=1)
        out = model.fit_transform(tri)
        
        #plot and save RDM matrices
        fig, (ax1,ax2) = plt.subplots(1,2,figsize=(8,3))
        title = 'Distance btw odors for {} in {} at {} (p={}) - Patient {} elec({})'.format(
            roi, freq[2:], exp[0], pval, su, str(elec))
        fig.suptitle(title)
        
        #subplot #1 Graph 2D 
        colors = ['red' if str(c) in su_odor_groups[su]['high'] else 'blue' for c in idx]
        markers = ['o' if c in context_su[su]['M'] else '*' for c in idx]
        for i, txt in enumerate(idx):
            ax1.scatter(out[i,0], out[i,1], c=colors[i], marker=markers[i],
                       label=['Rich' if colors[i]=='red' else 'Poor'])
            ax1.annotate('O'+str(txt), (out[i,0], out[i,1]))
        ax1.set_xlabel('component 1')
        ax1.set_ylabel('component 2')
        ax1.axis('equal')
        legend_elements = [Line2D([0], [0], marker='o', color='b', label='Poor_M'),
                           Line2D([0], [0], marker='*', color='b', label='Poor_C'),
                           Line2D([0], [0], marker='o', color='r', label='Rich_M'),
                           Line2D([0], [0], marker='*', color='r', label='Rich_C')]
        ax1.legend(handles=legend_elements, loc='best')
        
        #subplot #1 Graph 2D 
        cmap = cm.get_cmap('jet', 30)
        cax = ax2.imshow(tri, interpolation="none", cmap=cmap)
        ax2.set_xticks(np.arange(n_od))
        ax2.set_yticks(np.arange(n_od))
        ax2.set_xticklabels(idx,fontsize=11)
        ax2.set_yticklabels(idx,fontsize=11)
        plt.colorbar(cax)
        plt.subplots_adjust(top=0.92)
#         plt.tight_layout()

        if not exists(join(pathdata,'rdm_{}').format(exp[0],pval)):
            makedirs(join(pathdata,'rdm_{}').format(exp[0],pval))
                      
        plt.savefig(savename.format(exp[0],pval,freqs[0][2:],roi,exp[0],su, str(elec),pval))
        plt.savefig(savename2.format(exp[0],pval,freqs[0][2:],roi,exp[0],su, str(elec),pval))
        plt.clf()
        plt.close()

-> Olfacto loaded


In [27]:
"""
Plot TPSim matrices (RDM) for all odors by subject // AVERAGE BY ROI and SUBJECT
Plot Graph 2D representations of RDM matrices
"""
from brainpipe.system import study
from os.path import join
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib import cm
import seaborn as sns 
from matplotlib.patches import Patch
from sklearn.manifold import MDS
import glob

###############################################################################
st = study('Olfacto')
pathdata = join(st.path, 'figure/TPSim_LDA_{}_by_cond_6freqs_3s_dissim/npy_figs/')
path_npz = join(st.path,'feature/TPSim_{}_By_Odor_By_Cond/')
path_pow = join(path_npz, 'similarity_matrix_btw/TPS_pearson_{}_{}_btw_odors_mean.npz')
savename = join(pathdata, 'rdm_{}/{}_{}_{}_{}_p={}_mean_rdm.png')
savename2 = join(pathdata, 'rdm_{}/{}_{}_{}_{}_p={}_mean_rdm.pdf')
###############################################################################
exp, pval = 'Enc', '0.01' #Ret, Enc
freqs = ['0_theta']#, '1_alpha', '2_beta', '3_gamma']
###############################################################################

files = glob.glob(pathdata.format(exp[0])+freqs[0][2:]+'*'+pval+'_tps.csv')

for fi in files:
    roi = fi.split('_')[13]
    df = pd.read_csv(fi)
    subjects = df['subjects']
    
    for su in subjects:
        chan_su = df['channels'].loc[df['subjects']==su]
        OUT_all, TRI_all = np.array([]), np.array([])
        for chan in chan_su:
            mat = np.load(path_pow.format(exp,su,freqs[0]),allow_pickle=True)
            mat_chans = mat['channels']
            elec_id = [i for i,c in enumerate(mat_chans) if c == chan]
            combs, tps = mat['comb'], mat['tps'][elec_id,:]
            n_od = len(np.unique(combs))
            idx = list([combs[0,0]])+list([o for o in combs[:n_od-1,1]])
            tri = np.zeros((n_od, n_od))
            tri[np.triu_indices(n_od, 1)] = tps
            tri[np.tril_indices(n_od, -1)] = tri.T[np.tril_indices(n_od, -1)]
            model = MDS(n_components=2, dissimilarity='precomputed', random_state=1)
            out = model.fit_transform(tri)[np.newaxis]
            tri = tri[np.newaxis]
            TRI_all = np.concatenate((TRI_all,tri),axis=0) if np.size(TRI_all) else tri
            OUT_all = np.concatenate((OUT_all,out),axis=0) if np.size(OUT_all) else out
        out_m = np.mean(OUT_all,axis=0)
        tri_m = np.mean(TRI_all, axis=0)

        #plot and save RDM matrices
        fig, (ax1,ax2) = plt.subplots(1,2,figsize=(8,3))
        title = 'Distance btw odors for {} in {} at {} (p={}) - Patient {}'.format(
            roi, freq[2:], exp[0], pval, su)
        fig.suptitle(title)

        #subplot #1 Graph 2D 
        colors = ['red' if str(c) in su_odor_groups[su]['high'] else 'blue' for c in idx]
        markers = ['o' if c in context_su[su]['M'] else '*' for c in idx]
        for i, txt in enumerate(idx):
            ax1.scatter(out_m[i,0], out_m[i,1], c=colors[i], marker=markers[i],
                       label=['Rich' if colors[i]=='red' else 'Poor'])
            ax1.annotate('O'+str(txt), (out_m[i,0], out_m[i,1]))
        ax1.set_xlabel('component 1')
        ax1.set_ylabel('component 2')
        ax1.axis('equal')
        legend_elements = [Line2D([0], [0], marker='o', color='b', label='Poor_M'),
                           Line2D([0], [0], marker='*', color='b', label='Poor_C'),
                           Line2D([0], [0], marker='o', color='r', label='Rich_M'),
                           Line2D([0], [0], marker='*', color='r', label='Rich_C')]
        ax1.legend(handles=legend_elements, loc='best')

        #subplot #1 Graph 2D 
        cmap = cm.get_cmap('jet', 30)
        cax = ax2.imshow(tri_m, interpolation="none", cmap=cmap)
        ax2.set_xticks(np.arange(n_od))
        ax2.set_yticks(np.arange(n_od))
        ax2.set_xticklabels(idx,fontsize=11)
        ax2.set_yticklabels(idx,fontsize=11)
        plt.colorbar(cax)
        plt.subplots_adjust(top=0.92)
    #         plt.tight_layout()

        if not exists(join(pathdata,'rdm_{}').format(exp[0],pval)):
            makedirs(join(pathdata,'rdm_{}').format(exp[0],pval))

        plt.savefig(savename.format(exp[0],pval,freqs[0][2:],roi,exp[0],su,pval))
        plt.savefig(savename2.format(exp[0],pval,freqs[0][2:],roi,exp[0],su,pval))
        plt.clf()
        plt.close()

-> Olfacto loaded


In [34]:
"""
Check whether there is difference btw EARLY and LATE similarities during E and R
for High and Low conditions independantly
"""
from brainpipe.system import study
from os.path import join
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
from mne.stats import bonferroni_correction, fdr_correction
import glob

###############################################################################
st = study('Olfacto')
pathdata = join(st.path, 'figure/TPSim_LDA_{}_by_cond_6freqs_3s_dissim/npy_figs/')
path_npz = join(st.path,'feature/TPSim_{}_By_Odor_By_Cond/')
path_pow = join(path_npz, 'rdm_thgh_time/TPS_spear_{}_{}_{}_{}_wth_cond_first_vs_all.npz')
path_chans = join(st.path, 'feature/TPSim_power_data/Power_all_elecs_E_R_by_odor/')
chans_path = join(path_pow, '{}_odor{}_{}_pow{}.npz')
savename = join(pathdata, '{}_{}_{}_p={}_time.png')
savename2 = join(pathdata, '{}_{}_{}_p={}_time.pdf')
###############################################################################
exp, pval = 'Enc', '0.01' #Ret, Enc
freqs = ['theta']#, 'alpha', 'beta', 'gamma']
conds, wins = ['low','high'], ['early','late']
###############################################################################

files = glob.glob(pathdata.format(exp[0])+freqs[0]+'*'+pval+'_tps.csv')

for fi in files:
    freq,roi = freqs[0], fi.split('_')[-4]
    df = pd.read_csv(fi)
    subjects, channels = df['subjects'], df['channels']
        
    Tvals_l, pvals_l, Tvals_h, pvals_h = [],[],[],[]
    Tvals_0, pvals_0, Tvals_1, pvals_1 = [],[],[],[]
    early_l, early_h, late_l, late_h = [],[],[],[]
    for su, chan in zip(subjects,channels):
        mat_chans = np.load(path_pow.format(exp,su,'low',freq,'early'))['channels']
        elec_id = [i for i,c in enumerate(mat_chans) if c == chan][0]
        rdm0 = np.load(path_pow.format(exp,su,'low',freq,'early'))['rdm'][elec_id,:]
        rdm1 = np.load(path_pow.format(exp,su,'low',freq,'late'))['rdm'][elec_id,:]
        rdm2 = np.load(path_pow.format(exp,su,'high',freq,'early'))['rdm'][elec_id,:]
        rdm3 = np.load(path_pow.format(exp,su,'high',freq,'late'))['rdm'][elec_id,:]
        
        T0, p0 = stats.ttest_ind(rdm0,rdm1)
        T_e, p_e = stats.ttest_ind(rdm0, rdm2)
        T_l, p_l = stats.ttest_ind(rdm1, rdm3)
        T1, p1 = stats.ttest_ind(rdm2,rdm3)
        Tvals_l.append(T0), pvals_l.append(p0)
        Tvals_h.append(T1), pvals_h.append(p1)
        Tvals_0.append(T_e), pvals_0.append(p_e)
        Tvals_1.append(T_l), pvals_1.append(p_l)
        early_l.append(np.mean(rdm0)), early_h.append(np.mean(rdm2))
        late_l.append(np.mean(rdm1)), late_h.append(np.mean(rdm3))
    
    T_low, p_low = stats.ttest_rel(early_l,late_l)
    T_high, p_high = stats.ttest_rel(early_h,late_h)
    T_hl_e, p_hl_e = stats.ttest_rel(early_l, early_h)
    T_hl_l, p_hl_l = stats.ttest_rel(late_l, late_h)
    df['tps_early_low'], df['tps_late_low'] = early_l, late_l
    df['Tvals_low'], df['pvals_low'] = Tvals_l, pvals_l
    df['pvals_low_b'], df['pvals_low_fdr'] = bonferroni_correction(pvals_l)[1], fdr_correction(pvals_l)[1]
    df['tps_early_high'], df['tps_late_high'] = early_h, late_h
    df['Tvals_high'], df['pvals_high'] = Tvals_h, pvals_h
    df['pvals_high_b'], df['pvals_high_fdr'] = bonferroni_correction(pvals_h)[1], fdr_correction(pvals_h)[1]
    df.to_csv(fi, index=False)
    
    nsig_l = np.sum(np.array(pvals_l) < 0.05)
    nsig_h = np.sum(np.array(pvals_h) < 0.05)
    print('High/Low - early', T_hl_e, p_hl_e)
    print('High/Low - late', T_hl_l, p_hl_l)
    n_subj = len(np.unique(subjects))
    print(freq,roi,'For Low : {}/{} elecs sig in {} subjects'.format(nsig_l,len(df),n_subj))
    print('Group level stat : T={}, p={}'.format(round(T_low,2),round(p_low,3)))
    #print(df[['tps_early_low','tps_late_low','Tvals_low','pvals_low_b']])
    df_print2 = df[['subjects','elecs_num','tps_early_low','tps_late_low','Tvals_low','pvals_low']]
    df_print2 = df_print2.loc[(df_print2['pvals_low']<0.05)]
    print(df_print2)
    print(freq,roi,'For High : {}/{} elecs sig in {} subjects'.format(nsig_h,len(df),n_subj))
    print('Group level stat : T={}, p={}'.format(round(T_high,2),round(p_high,3)))
    df_print = df[['subjects','elecs_num','tps_early_high','tps_late_high','Tvals_high','pvals_high']]
    df_print = df_print.loc[(df_print['pvals_high']<0.05)]
    print(df_print)
    
    fig, (ax1,ax2) = plt.subplots(1,2,figsize=(8,3))
    title = 'Distance btw odors from LOW and HIGH in {} in {} at {} (p={})'.format(
        roi, freq, exp[0], pval, su, str(elec))
    fig.suptitle(title)
     
    x0, x1 = [np.mean(early_l),np.mean(late_l)], [np.mean(early_h), np.mean(late_h)]
    sd0, sd1 = [np.std(early_l),np.std(late_l)], [np.std(early_h), np.std(late_h)] 
    
    ax1.bar(np.arange(len(wins)), x0, width=0.5, yerr=sd0)
    ax1.set_ylabel('Similarity (r)')
    ax1.set_xticks(np.arange(len(wins)))
    ax1.set_xticklabels(wins)
    
    ax2.bar(np.arange(len(wins)), x1, width=0.5, yerr=sd1)
    ax2.set_ylabel('Similarity (r)')
    ax2.set_xticks(np.arange(len(wins)))
    ax2.set_xticklabels(wins)
    plt.subplots_adjust(top=0.92)
    plt.savefig(savename.format(exp[0],freq,roi,exp,pval))
    plt.savefig(savename2.format(exp[0],freq,roi,exp,pval))
    plt.clf()
    plt.close()

-> Olfacto loaded
High/Low - early 4.921012848066067 0.0006039493714532573
High/Low - late 6.681639633172057 5.4884909192601494e-05
theta aHC For Low : 0/11 elecs sig in 4 subjects
Group level stat : T=2.15, p=0.057
Empty DataFrame
Columns: [subjects, elecs_num, tps_early_low, tps_late_low, Tvals_low, pvals_low]
Index: []
theta aHC For High : 1/11 elecs sig in 4 subjects
Group level stat : T=0.34, p=0.743
  subjects  elecs_num  tps_early_high  tps_late_high  Tvals_high  pvals_high
1     FERJ         11        0.845547        0.55466     2.01941    0.048932
High/Low - early 2.298226995267543 0.1483284553598533
High/Low - late 3.2105533876689942 0.08485002226896932
theta IFG For Low : 0/3 elecs sig in 2 subjects
Group level stat : T=0.31, p=0.784
Empty DataFrame
Columns: [subjects, elecs_num, tps_early_low, tps_late_low, Tvals_low, pvals_low]
Index: []
theta IFG For High : 0/3 elecs sig in 2 subjects
Group level stat : T=1.63, p=0.244
Empty DataFrame
Columns: [subjects, elecs_num, tps_ea

In [41]:
"""
Check distance btw High and Low odors at EARLY and LATE E and R
"""
from brainpipe.system import study
from os.path import join
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
from mne.stats import bonferroni_correction, fdr_correction
import glob

###############################################################################
st = study('Olfacto')
pathdata = join(st.path, 'figure/TPSim_LDA_{}_by_cond_6freqs_3s_dissim/npy_figs/')
path_npz = join(st.path,'feature/TPSim_{}_By_Odor_By_Cond/')
path_pow = join(path_npz, 'rdm_thgh_time/TPS_pearson_{}_{}_{}_Low_High_med_split.npz')

savename = join(pathdata, '{}_{}_{}_p={}_high_low_time_med_split.png')
savename2 = join(pathdata, '{}_{}_{}_p={}_high_low_time_med_split.pdf')
###############################################################################
exp, pval = 'Enc', '0.01' #Ret, Enc
freqs = ['theta']#, 'alpha', 'beta', 'gamma']
wins = ['early','late']
###############################################################################

files = glob.glob(pathdata.format(exp[0])+freqs[0]+'*'+pval+'_tps.csv')

for fi in files:
    freq, roi = freqs[0],fi.split('_')[-4]
    df = pd.read_csv(fi)
    subjects, channels = df['subjects'], df['channels']
    
    Tvals, pvals, early, late = [],[],[],[]
    for su, chan in zip(subjects,channels):
        mat_chans = np.load(path_pow.format(exp,su,freq,'early'))['channels']
        id_elec = [i for i,c in enumerate(mat_chans) if c == chan][0]
        rdm0 = np.load(path_pow.format(exp,su,freq,'early'))['rdm'][id_elec,:]
        rdm1 = np.load(path_pow.format(exp,su,freq,'late'))['rdm'][id_elec,:]
        
        T0, p0 = stats.ttest_ind(rdm0,rdm1)
        Tvals.append(T0), pvals.append(p0)
        early.append(np.mean(rdm0)), late.append(np.mean(rdm1))

    T_tot, p_tot = stats.ttest_rel(early,late)
    df['tps_early_btw'], df['tps_late_btw'] = early, late
    df['Tvals_btw'], df['pvals_btw'] = Tvals, pvals
    df['pvals_btw_b'], df['pvals_btw_fdr'] = bonferroni_correction(pvals)[1], fdr_correction(pvals)[1]
    df_print = df[['subjects','elecs_num','tps_early_btw','tps_late_btw','Tvals_btw','pvals_btw']]
    df_print = df_print.loc[(df_print['pvals_btw']<0.05)]
    print(df_print)
    df.to_csv(fi, index=False)
    
    nsig = np.sum(bonferroni_correction(pvals)[1]<0.05)
    n_subj = len(np.unique(subjects))
    print(freq,roi,'Sig early/late : {}/{} elecs sig in {} subjects'.format(nsig,len(df),n_subj))
    print('Group level stat : T={}, p={}'.format(round(T_tot,2),round(p_tot,3)))
       
    plt.figure()
    title = 'Distance btw LOW and HIGH odors in {} in {} at {} (p={})'.format(
        roi, freq, exp[0], pval, su, str(elec))
    plt.suptitle(title)
    
    x0 = [np.mean(early),np.mean(late)]
    sd0 = [stats.sem(early),stats.sem(late)]
    
    plt.bar(np.arange(len(wins)), x0, width=0.5, yerr=sd0)
    plt.ylabel('Similarity (r)')
    plt.xticks(np.arange(len(wins)), labels=wins)
    
    plt.savefig(savename.format(exp[0],freq,roi,exp,pval))
    plt.savefig(savename2.format(exp[0],freq,roi,exp,pval))
    plt.clf()
    plt.close()

-> Olfacto loaded
Empty DataFrame
Columns: [subjects, elecs_num, tps_early_btw, tps_late_btw, Tvals_btw, pvals_btw]
Index: []
theta Ins Sig early/late : 0/4 elecs sig in 3 subjects
Group level stat : T=1.02, p=0.384
  subjects  elecs_num  tps_early_btw  tps_late_btw  Tvals_btw  pvals_btw
0     LEFC         21       1.039282      0.873944   3.459599   0.000607
theta olf Sig early/late : 1/2 elecs sig in 2 subjects
Group level stat : T=0.07, p=0.958
Empty DataFrame
Columns: [subjects, elecs_num, tps_early_btw, tps_late_btw, Tvals_btw, pvals_btw]
Index: []
theta pPirT Sig early/late : 0/2 elecs sig in 2 subjects
Group level stat : T=0.61, p=0.652
  subjects  elecs_num  tps_early_btw  tps_late_btw  Tvals_btw  pvals_btw
0     FERJ          6       1.228118      0.982979   3.101361   0.002401
1     FERJ         11       0.892565      0.610453   3.437379   0.000808
4     LEFC          5       1.065682      0.956168   2.389353   0.017398
6     LEFC          9       0.999354      0.901319   2.0

In [ ]:
"""
Check whether the strength of TPSim values (reinstatement or retrieval) is related to 
power values during encoding in aHC
Correlations by electrode and patient for 2 sig patients (LEFC & PIRJ)
We take average values by odor
"""

from os import listdir
from scipy.stats import spearmanr
import seaborn as sns 

st = study('Olfacto')
exp, freq = 'E', 'theta'
fnames = ['delta', 'theta', 'alpha', 'beta', 'low_gamma', 'high_gamma']
i_freq = [i for i,f in enumerate(fnames) if f == freq]
path_tps = join(st.path, 'feature/TPSim_Ret_By_Odor_By_Cond/TPS_by_odor/')
path_df = join(st.path, 'figure/TPSim_LDA_E_by_cond_3s/')
dfname = join(path_df, '2_all_subjects_info_elecs_AUC_Signif_th_0.001_mean_corr_all_11.csv')
path_pow = join(st.path, 'feature/TPSim_power_data/Power_all_elecs_E_R_by_odor/')
pow_name = join(path_pow, '{}_odor{}_{}_pow{}.npz')
###############################################################################
#dict_ = {'FERJ':["b2-b1","b6-b5"], 'PIRJ':["b'4-b'3"], 'VACJ':["b3-b2"]} #aHC
#dict_ = {'FERJ':["b2-b1","b6-b5","b3-b2","b'5-b'4"],
#         'PIRJ':["b'4-b'3"], 'VACJ':["b3-b2","b'2-b'1"], 'SEMC':["b5-b4"]} #aHC 10e-2
#dict_ = {'FERJ':["o9-o8","o10-o9"], 'SEMC':["j2-j1"], 'VACJ':["k'13-k'12"]} #IFG
#dict_ = {'FERJ':["o9-o8","o10-o9","o8-o7"], 'SEMC':["j2-j1"], 'VACJ':["k'13-k'12"]} #IFG 10e-2
#dict_ = {'LEFC':["o4-o3"], 'SEMC':["o5-o4","o6-o5"], 'PIRJ':["o5-o4"]} #OFC
#dict_ = {'LEFC':["o4-o3","o6-o5"], 'SEMC':["o5-o4","o6-o5","o7-o6"], 'PIRJ':["o5-o4"]} #OFC 10e-2
dict_ = {'FERJ':["a3-a2","j2-j1"], 'VACJ':["d'2-d'1"]} #PirC 10e-2
###############################################################################
rois = ['aHC']
odors_su = {'CHAF': {5:12,7:68,8:36,9:96,1:6,2:2,3:68,4:8},
            'LEFC': {1:4,2:0,3:6,4:12,14:96,15:2,16:4,17:68},
            'PIRJ': {4:36,9:2,1:4,18:32,6:34,5:4,7:68}, #missing odor 15
            'VACJ': {14:6,15:64,16:68,17:8,10:6,11:4,12:4,13:40},
            'SEMC': {10:2,11:6,12:6,13:6,5:8,7:4,8:8,9:10},
            'FERJ': {16:6,17:6,5:8,7:6,12:8,13:8,2:6,1:10}}

ALL_TPS, ALL_POW = [], []
for su in elecs_tps:
    for elec in elecs_tps[su]:
        all_pow, all_tps = [], []
        for od in odors_su[su]:
            tps_od = np.load(path_tps+'TPS_spear_{}_odor_{}_{}_3s.npz'.format(su,
                                                                od,freq))['tps'][elec,:]
            mean_tps = np.mean(tps_od)
            
            for roi in rois:
                df = pd.read_csv(dfname)
                labels = df.loc[df['subjects'] == su]['s_Mai_RL'].values
                idx = np.where(labels == roi)[0]
                pow_roi = np.load(pow_name.format(su,od,'E',freq))['xpow'][idx,17:47,:]    
                
                mean_pow = np.mean(pow_roi)
#                 mean_pow = np.mean(pow_roi,axis=(1,2))
                all_tps.append(mean_tps)
#                 all_tps.append(np.repeat(mean_tps,len(idx)))
                all_pow.append(mean_pow)
#                 ALL_TPS.append(np.repeat(mean_tps,len(idx)))
#                 ALL_POW.append(mean_pow)
#         R,p = spearmanr(np.concatenate(all_pow),np.concatenate(all_tps))
        R,p = pea(all_pow,all_tps)
        print(su, R, p)
#         sns.regplot(x=np.concatenate(all_pow), y=np.concatenate(all_tps));
        sns.regplot(x=all_pow, y=all_tps);
# R_all, p_all = spearmanr(np.concatenate(ALL_POW),np.concatenate(ALL_TPS))
# print('ALL', R_all, p_all)
# sns.regplot(x=np.concatenate(ALL_POW),y=np.concatenate(ALL_TPS));